In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.12.0


In [2]:
BATCH_SIZE = 2
SEQ_LEN = 100 # Длина последовательности
EMB_SIZE = 16 # Размер векторного представления (эмбеддинга)

x = np.random.rand(BATCH_SIZE, SEQ_LEN, EMB_SIZE).astype(np.float32)
print(x.shape)

(2, 100, 16)


In [3]:
H_SIZE = 32
OUT_SIZE = 16

rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(H_SIZE, activation='relu', return_sequences=True),
    tf.keras.layers.Dense(OUT_SIZE)
])

Metal device set to: Apple M2


In [4]:
y = rnn(x)
print(y.shape)

(2, 100, 16)


In [6]:
class RNNCell(tf.keras.Model):
    def __init__(self, h_size):
        super().__init__()
        self.h_size = h_size
        self.fcXH = tf.keras.layers.Dense(self.h_size)
        self.fcHH = tf.keras.layers.Dense(self.h_size, use_bias=False) # bias не нужен, так как он есть в fcXH
        
    def call(self, x, h):
        h = tf.nn.relu(self.fcXH(x) + self.fcHH(h))
        return h

In [7]:
# Рекурентная нейросеть
class RNN(tf.keras.Model):
    def __init__(self, h_size, out_size):
        super().__init__()
        self.h_size = h_size
        self.rnn_cell = RNNCell(h_size)
        self.fcHY = tf.keras.layers.Dense(out_size)
        
    def call(self, x_all):
        batch, length, emb _size = x.shape
        h = tf.zeros((batch, self.h_size))
        y_all = []
        
        for i in range(length):
            h = self.rnn_cell(x_all[:, i  :], h)
            y = self.fcHY(h)
            y_all.append(y)
            
        y_all = tf.transpose(tf.stack(y_all), [1, 0, 2])
        return y_all
    
rnn_my = RNN(H_SIZE, OUT_SIZE)

In [8]:
y = rnn_my(x)
print(y.shape)

(2, 100, 16)


In [9]:
# Перед тем, как что-то присваивать в параметры модели, нужно чтобы они создались.
# Для этого можно вызвать либо инференс с каким-то входом, либо model.build(...)

rnn_my.rnn_cell.fcXH.kernel = rnn.layers[0].weights[0]
rnn_my.rnn_cell.fcHH.kernel = rnn.layers[0].weights[1]
rnn_my.rnn_cell.fcXH.bias = rnn.layers[0].weights[2]

rnn_my.fcHY.kernel = rnn.layers[1].weights[0]
rnn_my.fcHY.bias = rnn.layers[1].weights[1]

y = rnn(x)
y_my = rnn_my(x)

print(np.max(np.abs(y.numpy() - y_my.numpy())))

4.7683716e-07
